# CORD-19 Metadata: Exploration, Cleaning, Analysis, and Streamlit App

This notebook walks through the full assignment: downloading `metadata.csv` from the CORD-19 dataset (or using a local copy), performing basic exploration, cleaning and preparation, analysis and visualizations, and generating a simple Streamlit app to explore the results.

Notes: the `metadata.csv` file can be large. The notebook contains a `sample` mode that reads only the first N rows or uses `chunksize`. If you have limited RAM, use the sampling cells.

In [ ]:
# Part 0: Imports and helper functions
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from wordcloud import WordCloud
sns.set(style='whitegrid')
%matplotlib inline

In [ ]:
# Configuration: set a metadata URL or local path.
# If you already downloaded metadata.csv, set METADATA_PATH to that file.
# Otherwise, set METADATA_URL to download directly.
METADATA_URL = None  # e.g. 'https://ai2-semanticscholar-cord-19.s3.amazonaws.com/2020-03-13/metadata.csv'
METADATA_PATH = 'metadata.csv'  # local filename to save to / or load from
# Use sample_mode to only load first N rows for quick iteration. Set to None to load full file.
SAMPLE_MODE = 20000  # set to None to attempt full load; or an integer to read only that many rows
CHUNKSIZE = 100000  # used for streaming/processing large files if needed
print('Config: METADATA_PATH=', METADATA_PATH, 'SAMPLE_MODE=', SAMPLE_MODE)

In [ ]:
# Part 1: Download (optional) and load the data
import urllib.request

,
,
# If a URL is provided, try to download (wrapped for safety)
if METADATA_URL and not os.path.exists(METADATA_PATH):
    try:
        download_metadata(METADATA_URL, METADATA_PATH)
    except Exception as e:
        print('Download failed:', e)

# Loading function that supports sampling and chunked preview
def load_metadata(path, sample_mode=None, chunksize=None):
    if sample_mode is not None:
        print(f'Loading first {sample_mode} rows from {path}...')
        return pd.read_csv(path, nrows=sample_mode, dtype=str)
    elif chunksize is not None:
        print(f'Reading with chunksize={chunksize}; concatenating first 1 chunk for preview...')
        it = pd.read_csv(path, chunksize=chunksize, dtype=str)
        return next(it)
    else:
        print(f'Loading full file {path}...')
        return pd.read_csv(path, dtype=str)

# Load the data (this will error if the file does not exist locally)
try:
    df = load_metadata(METADATA_PATH, sample_mode=SAMPLE_MODE, chunksize=None)
    print('Loaded DataFrame with', df.shape[0], 'rows and', df.shape[1], 'columns')
except FileNotFoundError as e:
    print('File not found. Please download metadata.csv into the notebook working directory or set METADATA_URL to a valid URL.')
    raise

In [ ]:
# Quick look: first rows and data types
df.head()

In [ ]:
# DataFrame shape and info
print('Shape:', df.shape)
print('
Info:')
print(df.info())

# Missing values per column (top columns)
miss = df.isnull().sum().sort_values(ascending=False)
miss.head(20)

## Part 2: Cleaning and Preparation

We'll convert date columns, extract year, and compute an abstract word count. We will also identify columns with many missing values and create a cleaned subset.

In [ ]:
# Convert publish_time to datetime (safe conversion)
if 'publish_time' in df.columns:
    df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')
    df['year'] = df['publish_time'].dt.year
    print('publish_time converted; sample years:', df['year'].dropna().unique()[:10])
else:
    print('No publish_time column found in DataFrame.')

# Create abstract_word_count if abstract column exists
if 'abstract' in df.columns:
    df['abstract_word_count'] = df['abstract'].fillna('').apply(lambda t: len(str(t).split()))
    print('abstract_word_count created; summary:')
    print(df['abstract_word_count'].describe())
else:
    print('No abstract column present.')

In [ ]:
# Identify columns with many missing values
total = len(df)
missing_frac = (df.isnull().sum() / total).sort_values(ascending=False)
missing_frac.head(30)

In [ ]:
# Create a cleaned subset: keep important columns and drop columns with >80% missing
threshold = 0.8
cols_keep = missing_frac[missing_frac <= threshold].index.tolist()
print('Keeping', len(cols_keep), 'columns out of', len(df.columns))
df_clean = df[cols_keep].copy()

## Part 3: Analysis and Visualizations

We'll compute: publications by year, top journals, word frequencies in titles, and create plots including a word cloud.

In [ ]:
# Publications by year
if 'year' in df_clean.columns:
    year_counts = df_clean['year'].value_counts(dropna=True).sort_index()
    print(year_counts.head())
    plt.figure(figsize=(8,4))
    year_counts.plot(kind='bar')
    plt.title('Publications by Year')
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.tight_layout()
else:
    print('Year column not available for plotting.')

In [ ]:
# Top journals / sources
# CORD-19 sometimes uses 'journal' or 'journal' fields; try common column names
journal_col = None
for candidate in ['journal', 'journal_title', 'journal_ref', 'source_x', 'source_y']:
    if candidate in df_clean.columns:
        journal_col = candidate
        break

if journal_col:
    top_journals = df_clean[journal_col].fillna('Unknown').value_counts().head(20)
    plt.figure(figsize=(10,6))
    sns.barplot(y=top_journals.index, x=top_journals.values, palette='viridis')
    plt.title('Top publishing journals/sources')
    plt.xlabel('Count')
    plt.ylabel('Journal')
    plt.tight_layout()
    print('Using journal column:', journal_col)
else:
    print('No journal-like column found. Columns available:', df_clean.columns.tolist())

In [ ]:
# Most frequent words in titles (simple tokenization)
if 'title' in df_clean.columns:
    titles = df_clean['title'].fillna('').astype(str).str.lower()
    # basic cleaning and tokenization
    def tokenize(s):
        s = re.sub(r
, ' ', s)
        return [w for w in s.split() if len(w) > 2]
    all_words = Counter()
    for t in titles:
        all_words.update(tokenize(t))
    common_words = all_words.most_common(50)
    common_words[:20]
else:
    print('No title column available.')

In [ ]:
# Word cloud for titles (if pillow & wordcloud are installed)
if 'title' in df_clean.columns:
    text = ' '.join(df_clean['title'].dropna().astype(str).tolist())
    wc = WordCloud(width=800, height=400, background_color='white').generate(text.lower())
    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud of Titles')
else:
    print('Skipping word cloud: no title column')

## Part 4: Save cleaned data and create Streamlit app file
We'll save a cleaned CSV (`metadata_clean.csv`) and create a simple `app.py` that uses Streamlit to display the visualizations.

In [ ]:
# Save cleaned data for the app (downsample if sample_mode used)
out_clean = 'metadata_clean.csv'
df_clean.to_csv(out_clean, index=False)
print('Saved cleaned CSV to', out_clean)

In [ ]:
# Quick sample of cleaned data
df_clean.head()

## Part 5: Notes / Next Steps and Reflection

This notebook performed the requested tasks in a reproducible way and saved a cleaned CSV which the Streamlit app will use. See `README.md` for instructions to run the Streamlit app.

Potential improvements: more robust title tokenization and stopword removal (use NLTK or spaCy), temporal smoothing of publication counts, interactive plots (Plotly), and additional metadata joins (PMC / PubMed IDs).